In [2]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import openpyxl
import json
import sqlalchemy as sa
import urllib

САЙТ НАЦ БАНКА РК

In [ ]:
url = 'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut/excel'

query = '?beginDate=18.06.2024&endDate=25.06.2024&rates%5B%5D=1&rates%5B%5D=5&rates%5B%5D=6&rates%5B%5D=7&rates%5B%5D=16'

# 1 - AUD
# 5 - USD
# 6 - EUR
# 7 - CAD
# 16 - RUB

Шаг 1: Скачать и сохранить файл

In [62]:
# Получить текущую дату
end_date = datetime.now()
# Получить дату неделю назад
begin_date = end_date - timedelta(days=0)

# Преобразовать даты в нужный формат 'dd.mm.yyyy'
end_date_str = end_date.strftime('%d.%m.%Y')
begin_date_str = begin_date.strftime('%d.%m.%Y')

# Загрузка конфигурации базы данных
with open('config.json') as config_file:
    config = json.load(config_file)

currencies = list(config['currency_code'].keys())
rates = list(config['currency_code'].values())

In [63]:
url = 'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut/excel'
params = {
    'beginDate': begin_date_str,
    'endDate': end_date_str,
    'rates[]': rates
}

response = requests.get(url, params=params)

if response.status_code == 200:
    filename = 'exchange_rates.xlsx'
    file_path = f'C:\\Users\\Bim_User\\Downloads\\{filename}'
    
    # Сохранение файла
    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    print(f'File saved as {file_path}')
else:
    print(f'Failed to download file. Status code: {response.status_code}')

File saved as C:\Users\Bim_User\Downloads\exchange_rates.xlsx


Шаг 2: Считать Excel файл и подготовить данные

In [58]:
# Считать Excel файл в DataFrame
df = pd.read_excel(file_path, engine='openpyxl')

# Исключить ненужные столбцы и переименовать оставшиеся
currencies.insert(0, 'Date')
df = df[currencies]
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

df

,Date,AUD,USD,EUR,CAD,RUB
0,2024-06-18,299.24,452.92,484.99,329.66,5.12
1,2024-06-19,303.09,457.98,490.86,333.17,5.26
2,2024-06-20,306.75,459.75,494.14,335.39,5.54
3,2024-06-21,306.60,459.60,493.06,335.23,5.38
4,2024-06-22,309.18,464.72,496.32,339.39,5.40
5,2024-06-23,309.18,464.72,496.32,339.39,5.40
6,2024-06-24,309.18,464.72,496.32,339.39,5.40
7,2024-06-25,309.32,465.28,499.20,340.29,5.33


In [64]:
currencies

['AUD', 'USD', 'EUR', 'CAD', 'RUB']

In [66]:
df_melted = df.melt(id_vars=['Date'], value_vars=currencies, var_name='CurrencyCode', value_name='CurrencyValue')
df_melted.columns = ['DATE', 'CurrencyCode', 'CurrencyValue']
df_melted['DATE'] = pd.to_datetime(df_melted['DATE'], infer_datetime_format=True)

df_melted

,DATE,CurrencyCode,CurrencyValue
0,2024-06-18,AUD,299.24
1,2024-06-19,AUD,303.09
2,2024-06-20,AUD,306.75
3,2024-06-21,AUD,306.60
4,2024-06-22,AUD,309.18
5,2024-06-23,AUD,309.18
6,2024-06-24,AUD,309.18
7,2024-06-25,AUD,309.32
8,2024-06-18,USD,452.92
9,2024-06-19,USD,457.98


Шаг 3: Загрузить данные в MS SQL

In [22]:
def connect_to_db(connection_string):
    connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(connection_string)}"
    engine = sa.create_engine(connection_uri, fast_executemany=True, echo=True)
    #connection = engine.raw_connection()
    return engine

In [ ]:
# Загрузка конфигурации базы данных
with open('config.json') as config_file:
    config = json.load(config_file)

database_uri = config['connection_string']
engine = connect_to_db(database_uri)

# Загрузка данных в таблицу
df.to_sql('currency_history', engine, if_exists='append', index=False)
print("Data has been inserted successfully.")